## IMPLEMENTATION

In this section we will explore exactly how each API returns data of interest.

We will look at how to obtain 

In [1]:
import pandas as pd
import requests

## OPENAQ

In [2]:
oaq_endpoint = "https://api.openaq.org/v2/cities?country=MX"

with open("oaq_cities_data_mx.json", "wb") as fo, requests.get(oaq_endpoint) as r:
    fo.write(r.content)

In [3]:
oaq_endpoint = "https://api.openaq.org/v2/locations?city=Mexico City"

with open("oaq_data_mexico_city.json", "wb") as fo, requests.get(oaq_endpoint) as r:
    fo.write(r.content)

## WORLD AQI

In [4]:
with open("waqi_key.txt") as f:
    api_key = f.read().strip()

**NOTE**: we will NOT assume the end user has these libraries installed, thus making this a usable and useful notebook, instead of one that will break when the requisite libraries are not installed.  We should also put this in a `requirements.txt` file.

In [5]:
!pip install  ozon3

In [6]:
import ozon3 as ooo
o3 = ooo.Ozon3(api_key)

station = (19.334,-99.182)
data = o3.get_coordinate_air(*station)

In [7]:
data.columns

Index(['city', 'latitude', 'longitude', 'station', 'dominant_pollutant',
       'timestamp', 'timestamp_timezone', 'aqi', 'AQI_meaning',
       'AQI_health_implications', 'pm2.5', 'pm10', 'o3', 'co', 'no2', 'so2',
       'dew', 'h', 'p', 't', 'w', 'wg'],
      dtype='object')

In [12]:
data['pm2.5'], data['timestamp']

(0    95.0
 Name: pm2.5, dtype: float64,
 0    2024-01-11 11:00:00
 Name: timestamp, dtype: object)

## Compare measurements

Because the city is empty, we might want to know if we are
looking at the same station.

Let's compare the data to find out -- if it is the same station 
(maybe) the data will be the same.

In [9]:
oaq_endpoint = "https://api.openaq.org/v2/latest/1285351"

with requests.get(oaq_endpoint) as r:
    print(r.json()['results'][0])
    # df = pd.DataFrame.from_records(r.json()['results'][0])
    df = pd.json_normalize(r.json()['results'], max_level=1)
    df_m = pd.DataFrame.from_dict(df.measurements[0])
    
    df = pd.concat([df, df_m], axis=1).drop('measurements', axis=1)

{'location': 'SPARTAN - Mexico City', 'city': 'Mexico City', 'country': 'MX', 'coordinates': {'latitude': 19.334, 'longitude': -99.182}, 'measurements': [{'parameter': 'pm25', 'value': 26.9, 'lastUpdated': '2023-05-31T11:00:00+00:00', 'unit': 'µg/m³'}]}


In [10]:
df['value']

0    26.9
Name: value, dtype: float64

In [11]:
df['value'] == data['pm2.5']

0    False
dtype: bool

These things don't look the same.  Our analysis is done. $\Xi$